In [1]:
import requests

url_search = 'https://www.topagrar.com/?-type=standardbeitrag&tree_id=7697&doSearch=1&query=gülle&search=&tree.id=7538&sortfieldwithorder=date_desc&date_years=&filterbytree=0&page='
page_max = 530

resp = requests.get(url_search)

In [2]:
import shutil
from os import makedirs

topagrar_local = 'topagrar/topagrar_search/'

makedirs(topagrar_local, exist_ok = True)
shutil.rmtree(topagrar_local)
makedirs(topagrar_local, exist_ok = True)

In [3]:
with open('1.html', 'w') as wf:
    wf.write(resp.text)

    for pagenum in range(1, page_max):
        url = url_search + str(pagenum)
    #    print('Downloading', url)
        resp = requests.get(url)

        fname = topagrar_local + str(pagenum) + '.html'
    #    print('Saving to', fname)
        with open(fname, 'w') as wf:
            wf.write(resp.text)
            
print('\n')
print('done...')



done...


In [4]:
from glob import glob
from os.path import join

html_filenames = glob(join(topagrar_local, '*.html'))

In [5]:
topagrar_page = 'topagrar/topagrar_page/'

makedirs(topagrar_page, exist_ok = True)
shutil.rmtree(topagrar_page)
makedirs(topagrar_page, exist_ok = True)

In [6]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin

url_top = 'https://www.topagrar.com'

for hname in html_filenames:

    with open(hname, 'r') as rf:
        txt = rf.read()
        soup = BeautifulSoup(txt, 'lxml')

        for hed in soup.find_all('h2'):
            href = hed.find('a').attrs['href']
            url = urljoin(url_top, href)
    #        print('Downloading: ', url)
            resp = requests.get(url)

            fn = href.replace('/', '-').strip('-') + '.html'
            full_fname = join(topagrar_page, fn)
    #        print('Saving ', full_fname)
            with open(full_fname, 'w') as wf:
                wf.write(resp.text)
            
print('\n')
print('done...')



done...


In [7]:
import pandas as pd
import glob
import os.path
import datetime
import re

df_topagrar = pd.DataFrame()

for file_name in glob.glob(os.path.join(topagrar_page, '*.html')):
    
    with open(file_name) as html_file:
        soup = BeautifulSoup(html_file)
        
        for i in soup.select('main'):
            article_category = soup.find('li', class_='m-breadcrumb__item').text
        
            topic = soup.find('h2', class_='a-topic')
            if topic:
                article_topic = soup.find('h2', class_='a-topic').text
            else:
                article_topic = ('')
                
            article_webscraping = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    
            article_title = soup.find('h1').text
            article_title = re.sub("Plus", " ", article_title)
            article_title = re.sub("\n", " ", article_title)

            article_lead = soup.find('p', class_='a-lead').text
            
            article_publication = soup.find('time', class_='m-metadata__item').text
            article_publication = re.sub("\n", " ", article_publication)
            article_publication = pd.to_datetime(article_publication)

            premium_text = soup.find('div', class_='paywall-full-content')
            if premium_text:
               article_text = soup.find('div', class_='paywall-full-content').text
            else:
               article_text = soup.find('div', class_='l-articleview__pagetext').text
            
            article_text = re.sub("\n", " ", article_text)
                
            premium_status = soup.find('div', class_='paywall-full-content')
            if premium_status:
                article_status = ('Plus')
            else:
                article_status = ('Free')
                
            article_url = soup.find("meta",  property="og:url")
            article_url = article_url['content']
            
            article_corpus = article_title + article_lead + article_text
            
            df_topagrar = df_topagrar.append(pd.DataFrame([[article_url, article_webscraping,
                                                            article_status, article_topic,
                                                            article_title, article_lead,
                                                            article_text, article_corpus,
                                                            article_category, article_publication]],
                                                          columns=['URL', 'WebScraping',
                                                                   'Status', 'Topic',
                                                                   'Title', 'Lead',
                                                                   'Text', 'Corpus',
                                                                   'Category', 'Publication']))
            
print('\n')
print('done...')



done...


In [8]:
file_name

'topagrar/topagrar_page/energie-aus-dem-heft-biogas-nrw-regel-wird-zum-aergernis-9677272.html?utm_content=search.html'

In [9]:
df_topagrar.reset_index(drop = True, inplace = True)
#df_topagrar

In [10]:
df_topagrar.to_csv('topagrar/df_topagrar_20200605.csv', sep = ';', index = False)

In [11]:
import pandas as pd

df = pd.read_csv('topagrar/df_topagrar_20200605.csv', sep = ';')
df

,URL,WebScraping,Status,Topic,Title,Lead,Text,Corpus,Category,Publication
0,https://www.topagrar.com/acker/aus-dem-heft/ac...,2020-06-05 17:50:00,Plus,NaN,"Gülle-Strip Till – weniger Erosion, mehr Ertra...",Das Strip Till-Verfahren zu Mais in Kombinatio...,Das Strip Till-Verfahren zu Mais in Kom...,"Gülle-Strip Till – weniger Erosion, mehr Ertra...",Acker & Agrarwetter,2015-03-25
1,https://www.topagrar.com/energie/news/baywa-in...,2020-06-05 17:50:00,Free,NaN,BayWa investiert in Biogas,Ein Tochterunternehmen der BayWa AG (BayWa r.e...,Ein Tochterunternehmen der BayWa AG (BayWa r...,BayWa investiert in Biogas Ein To...,Energie,2010-07-14
2,https://www.topagrar.com/technik/aus-dem-heft/...,2020-06-05 17:50:00,Plus,top agrar Test,Tieflader für Flüssiges,NaN,Der fast eckige Stahlbehälter ist das Besonde...,Tieflader für Flüssiges Der fas...,Technik,2016-09-21
3,https://www.topagrar.com/acker/news/positionsp...,2020-06-05 17:50:01,Plus,Bauernverband Schleswig-Holstein,Positionspapier: So sollen Bauern künftig Acke...,Der Bauernverband Schleswig-Holstein hat seine...,Die Aussagen des schleswig-holsteinischen Bau...,Positionspapier: So sollen Bauern künftig Acke...,Acker & Agrarwetter,2019-07-23
4,https://www.topagrar.com/schwein/aus-dem-heft/...,2020-06-05 17:50:01,Plus,NaN,Unser Ziel sind 350 Sauen im Kombibetrieb ...,NaN,"Um in der Mast Spitzenleistungen zu erzielen,...",Unser Ziel sind 350 Sauen im Kombibetrieb ...,Schwein,2004-01-11
...,...,...,...,...,...,...,...,...,...,...
3093,https://www.topagrar.com/schwein/aus-dem-heft/...,2020-06-05 17:54:15,Plus,NaN,"Schweine, Schule, Speckweckerl",NaN,Spielerisch lernen Grundschüler auf dem Betri...,"Schweine, Schule, Speckweckerl ...",Schwein,2012-12-19
3094,https://www.topagrar.com/schwein/news/kadaver-...,2020-06-05 17:54:15,Free,NaN,Kadaver auf dem Feld empören Bürger,In Österreich steht derzeit ein Schweinebetrie...,Im österreichischen Bezirk Klagenfurt-Land st...,Kadaver auf dem Feld empören Bürger ...,Schwein,2018-12-04
3095,https://www.topagrar.com/acker/news/niedersach...,2020-06-05 17:54:15,Free,Düngeverschärfung,Niedersachsen stellt Nitrat- und Phosphat-Kuli...,"„Jetzt wissen wir, wie die Karte aussehen wird...",Die Gebietskulissen der nitrat- und phosphats...,Niedersachsen stellt Nitrat- und Phosphat-Kuli...,Acker & Agrarwetter,2019-10-09
3096,https://www.topagrar.com/management-und-politi...,2020-06-05 17:54:15,Free,NaN,Umweltschützer prangern Gülle-Unfälle an ...,Der BUND prangert in einer so genannten „Chron...,Der Bund für Umwelt und Naturschutz Deutschla...,Umweltschützer prangern Gülle-Unfälle an ...,Politik,2016-09-29
